In [1]:
import math

import geojson
import requests
import tqdm

In [2]:
#GLOSSIS
dataset = 'glossis'
locations_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/locations'
nodes_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/nodes'
observation_types_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/observationTypes'
timeseries_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/timeseries'

In [3]:
# GLOFFIS
dataset = 'gloffis'
locations_url = 'http://tw-151.xtr.deltares.nl:8081/FewsWebServices/rest/digitaledelta/2.0/locations'
nodes_url = 'http://tw-151.xtr.deltares.nl:8081/FewsWebServices/rest/digitaledelta/2.0/nodes'
observation_types_url = 'http://tw-151.xtr.deltares.nl:8081/FewsWebServices/rest/digitaledelta/2.0/observationtypes'
timeseries_url = 'http://tw-151.xtr.deltares.nl:8081/FewsWebServices/rest/digitaledelta/2.0/timeseries'

In [4]:
url = locations_url
print(requests.get(locations_url))
resp = requests.get(url).json()

<Response [200]>


In [5]:
# get the first request and inspect it to get the paging information

def paging_iter(url):
    """iterate over all pages in a url"""
    print(requests.get(url))

    resp = requests.get(url).json()
    page_size = resp['paging']['maxPageSize']
    count = resp['paging']['totalObjectCount']
    # Now loop over all pages
    n = math.ceil(count / page_size)

    # store each json result
    # progress please
    for i in tqdm.tqdm_notebook(range(n)):
        # fire the request. It's not faster in parallel (tested with asyncio), so I keep it serial.
        resp = requests.get(url, dict(page=i + 1, pageSize=page_size))
        yield resp.json()
    

In [6]:
locations_resps = list(paging_iter(locations_url))

<Response [200]>


In [7]:
# collect information about the information per location
# one request per location
timeseries_resps = list(paging_iter(timeseries_url))

<Response [200]>


In [8]:
# collect all the features
features = []
for resp in locations_resps:
    results = resp['results']
    for feature in results:
        feature['id'] = feature['properties']['locationId']
        features.append(feature)        
    
features_by_id = {
    feature['id']: feature
    for feature 
    in features
}

timeseries = []
for resp in timeseries_resps:
    timeseries.extend(resp['results'])

In [20]:
all_quantities = []

for series in timeseries:
    id = series['location']['properties']['locationId']
    feature = features_by_id[id]
    quantities = feature['properties'].get('quantities', [])    
    if dataset == 'glossis':
        quantity = series['observationType']['quantity']
    elif dataset == 'gloffis':
        quantity = series['observationType']['parameterCode']       
    if quantity not in quantities:
        quantities.append(quantity)
    if quantity not in all_quantities:
        all_quantities.append(quantity)
    feature['properties']['quantities'] = quantities
    
print(all_quantities)

['Q.obs', 'Q.simulated', 'EPot', 'T', 'P']


In [21]:
serie = timeseries[0]
feature = features[0]
print(feature)

quantities = feature['properties'].get('quantities', [])
print(quantities)

quantity = series['observationType']['parameterCode']
print(quantity)

if quantity not in quantities:
        quantities.append(quantity)
        
print(quantities)

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [19.6019993, 44.9669991, 72.22]}, 'properties': {'node': {'url': 'http://tw-151.xtr.deltares.nl:8081/FewsWebServices/rest/digitaledelta/2.0/nodes/Deltares', 'id': 'Deltares', 'name': 'Deltares Digitale Delta'}, 'locationId': 'Sremska Mitrovica', 'locationName': 'Sremska Mitrovica', 'referenceLevel': 'm3/s', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'quantities': ['Q.obs', 'Q.simulated']}, 'id': 'Sremska Mitrovica'}
['Q.obs', 'Q.simulated']
Q.obs
['Q.obs', 'Q.simulated']


In [22]:
# Add Booleans for all quantities to each feature
for feature in features:
    for quantity in all_quantities:
        if quantity in feature["properties"]["quantities"]:
            feature["properties"][quantity] = True
        else:
            feature["properties"][quantity] = False
    features_by_id[feature['id']] = feature

In [23]:
# store the features
collection = geojson.FeatureCollection(features=list(features_by_id.values()))
with open('tw-151.geojson', 'w') as f:
    geojson.dump(collection, f)

In [ ]:
# Create tiles until level 12 or something.... with tippecanoe

# let's upload it
!mapbox upload siggyf.pl-tc012 pl-tc012.geojson